Exploring the how reagent contribute to gene counts. 

In [1]:
# %load ../start.py
# Load useful extensions
import os
import sys

# Activate the autoreload extension for easy reloading of external packages
%reload_ext autoreload
%autoreload 1

# Set up cashdir
from ipycache import CacheMagics
CacheMagics.cachedir = '../cachedir'

# Trun on the water mark
%reload_ext watermark
%watermark -u -d -v -g

# Load ipycache extension
%reload_ext ipycache
from ipycache import CacheMagics
CacheMagics.cachedir = '../cachedir'

# Add project library to path
sys.path.insert(0, '../../lcdb-wf/lib')
sys.path.insert(0, '../../lib/python')

# Set up references
import yaml
with open('../../config/config.yml') as fh:
    config = yaml.load(fh)

assembly = config['assembly']
tag = config['aligner']['tag']
REF = os.path.join(os.environ['REFERENCES_DIR'], assembly, tag)


last updated: 2017-09-13 

CPython 3.5.2
IPython 6.1.0
Git hash: f20b1ea9e0a95e9c9bcb262ed1f1b5b5e6ae8b02


In [2]:
# imports
import collections
import pandas as pd
import pybedtools
import HTSeq
import yaml

In [3]:
# import sample table
with open('../../config/config.yml') as fh:
    config = yaml.load(fh)
    
srr = 'SRR3486828'
stable = pd.read_table('../../config/sampletable.tsv')
mask = stable.samplename == srr
drsc = stable[mask].drsc.iloc[0]
fbgn = stable[mask].target_FBgn.iloc[0]

stable[mask]

,samplename,SRX,BioSample,GEO,drsc,target_FBgn,target_symbol,drsc_rep,rep,plate_id,well_id,plate_row,plate_column
948,SRR3486828,SRX1748824,SAMN04959159,GSM2145091,DRSC30750,FBgn0284249,Adf1,1,2,10,H4,H,4


In [4]:
drsc, fbgn

('DRSC30750', 'FBgn0284249')

In [5]:
sys.path.insert(0, '../../bin')

In [6]:
from drsc_adjust_count import *

In [7]:
drsc_bed = get_drsc_bed('../../output/drsc_coordinates.bed', drsc)

In [8]:
gene_bed = filter_gtf(config, fbgn)

In [9]:
gene_sub = gene_bed.subtract(drsc_bed)

In [13]:
gene_interval = build_interval_from_file(gene_bed.fn)

In [15]:
gene_sub_interval = build_interval_from_file(gene_sub.fn)

In [16]:
drsc_interval = build_interval_from_list(drsc_bed)

In [20]:
bam = os.path.join(
    '../../rnaseq-wf/data/rnaseq_samples/{srr}/{srr}.cutadapt.bam'.format(srr=srr)
)
counts, reads = count_algn(gene_interval, gene_sub_interval, drsc_interval, bam)

In [21]:
counts

{'drsc_count': 178, 'gene_count': 460, 'sub_count': 296}

In [22]:
reads

['>SRR3486828.7924951\n                GCTCTGTTTGTTTTTATTTTGATTGGTTTCTGCTACGAGGAAGGAACGTCG\n                +\n                CCCFFDDEHHFFBCEHBDB@EHHIGIHGHIG*?CFDD0?@HGE;?BFG8BB',
 '>SRR3486828.1004772\n                GCACGCTCTGTTTGTTTTTATTTTGATTGGTTTCTTCTACGAGGAA\n                +\n                @;@DB@D?FDFHAHGBACG@@CHHA8AEC@B=?CF<@FG0?:@66B',
 '>SRR3486828.4484954\n                GCTGTTAAGTATTTGGCGCGAACGCACGCTCTGTTTGTTTTTATTTTGATT\n                +\n                ?@;DADDDHDFDFIIIIBEGIIIIIEIBGGDHIGGIIGHIGGCEEHI@;CE',
 '>SRR3486828.6215833\n                GTTGTTAAGTATTTGGCGCGAACGCACGCTCTGTTTGTTTTTATTTTGATT\n                +\n                BBCFDDFFHCFHHJJJJIJJJJJIJJJJJJJJJBHHGGHIJJJJJJJHHHH',
 '>SRR3486828.35985\n                GTTGTTAAGTATTTGGCGCGAACGCACGCTCTGTTTGTTTTTTTTTTTTTT\n                +\n                @?@FDFDDFHFHHIJJFFHGIB@DGIDGG@FB>FFHGCGGGHHFCDDDDD@',
 '>SRR3486828.3217434\n                TTGCTAATTGTTGTTAAGTATTTGGCGCGAACGCACGCTCTGTTTGTTTTT\n               